# Test énergie backend API

---

## Objectif

Ce notebook simule des appels concurrents à l'API `/cable_temperature_consumption_simulation` pour évaluer l'empreinte énergétique backend selon la charge (nombre d'utilisateurs/minute).  
L'énergie affichée correspond à la somme des valeurs `co2_emissions` retournées par l'API pour chaque appel.

- **10 utilisateurs/minute** : 10 requêtes simultanées.
- **100 utilisateurs/minute** : 100 requêtes simultanées.
- **1000 utilisateurs/minute** : 1000 requêtes simultanées.
- **1000 utilisateurs/minute + cache** : 1000 requêtes avec le cache activé côté backend (si disponible).

Le tableau final permet de comparer l'impact énergétique de l'API selon la charge et l'utilisation éventuelle d'un cache.

---

## Interprétation

- **Énergie (kgCO2)** : somme des émissions de CO₂ rapportées par l'API pour tous les utilisateurs du scénario.
- Pour une analyse fine, adaptez le paramètre `use_cache` selon la logique de votre backend.

---

> **Note** : Ce notebook nécessite que le serveur FastAPI soit lancé et accessible à l'URL `http://localhost:8000`.
> Assurez-vous que le backend est opérationnel avant de l'exécuter.
> Voir [README.md](../README.md) pour les instructions de lancement du serveur.

## Imports

Ce notebook nécessite les bibliothèques suivantes :
- `threading`
- `requests`
- `codecarbon`
- `tabulate`

In [1]:
import threading

import requests
from codecarbon import EmissionsTracker
from tabulate import tabulate

## Configuration de l'API

Cette section configure l'URL de l'API et les paramètres

In [2]:
API_URL = "http://localhost:8000/cable_temperature_consumption_simulation"

payload = {
    "ambient_temperature": 25,
    "wind_speed": 2,
    "current_intensity": 300,
    "initial_cable_temperature": 25,
    "simulation_duration": 60,
    "time_step": 1e-1
}

## Vérification du serveur API

Cette section vérifie que le serveur FastAPI est bien lancé et accessible.

In [3]:
# Vérification que le serveur est bien lancé
try:
    r = requests.get("http://localhost:8000/health", timeout=3)
    if r.status_code != 200:
        raise Exception("Le serveur API ne répond pas correctement.")
except Exception as e:
    print("ERREUR : Le serveur FastAPI n'est pas lancé ou inaccessible.")
    print("Démarrez le backend avant d'exécuter ce notebook.")
    raise SystemExit(e)

## Création des fonctions de test

Cette section définit les fonctions nécessaires pour envoyer des requêtes POST à l'API et mesurer l'énergie consommée.

In [4]:
def sci(val):
    try:
        return f"{float(val):.10e}"
    except:
        return val


def send_post_request() -> float:
    """
    Envoie une requête POST à l'API pour simuler la consommation d'énergie.
    :return: L'énergie consommée en kgCO2.
    """
    res = requests.post(API_URL, params=payload)
    if res.status_code != 200:
        raise Exception(f"Erreur lors de l'appel à l'API : {res.status_code} - {res.text}")
    data = res.json()
    if "co2_emissions" not in data:
        raise Exception("La réponse de l'API ne contient pas 'co2_emissions'.")
    return data["co2_emissions"]


def run_users_test(nb_users: int, use_cache: bool = False) -> float:
    """
    Exécute un test avec un nombre donné d'utilisateurs simulés.
    :param nb_users: Nombre d'utilisateurs à simuler.
    :param use_cache: Indique si le cache doit être utilisé (optionnel).
    :return: L'énergie totale consommée en kgCO2.
    """
    payload["use_cache"] = use_cache  # Ajouter un paramètre pour le cache si nécessaire

    results = []
    threads = []

    def thread_func():
        try:
            val = send_post_request()
            results.append(val)
        except Exception:
            raise SystemExit(e)

    tracker = EmissionsTracker(measure_power_secs=1, save_to_file=False, log_level="warning")
    tracker.start()

    for _ in range(nb_users):
        thread = threading.Thread(target=thread_func)
        threads.append(thread)
        thread.start()

    for thread in threads:
        thread.join()

    co2_energy = sum(results)
    co2_energy += tracker.stop()
    return co2_energy


## Exécution des tests

Cette section exécute les tests pour différents nombres d'utilisateurs et affiche les résultats.

### Test avec 10 utilisateurs/minute

In [5]:
co2_energy_10 = run_users_test(10)
print(f"10 utilisateurs/minute : {sci(co2_energy_10)} kgCO2")

[codecarbon WARNING @ 16:53:37] Multiple instances of codecarbon are allowed to run at the same time.
[codecarbon WARNING @ 16:53:39] No CPU tracking mode found. Falling back on estimation based on TDP for CPU. 
 Windows OS detected: Please install Intel Power Gadget to measure CPU

[codecarbon WARNING @ 16:53:39] No CPU tracking mode found. Falling back on CPU constant mode.


10 utilisateurs/minute : 1.7442356479e-05 kgCO2


### Test avec 100 utilisateurs/minute

In [6]:
co2_energy_100 = run_users_test(100)
print(f"100 utilisateurs/minute : {sci(co2_energy_100)} kgCO2")

[codecarbon WARNING @ 16:53:57] Multiple instances of codecarbon are allowed to run at the same time.
[codecarbon WARNING @ 16:53:59] No CPU tracking mode found. Falling back on estimation based on TDP for CPU. 
 Windows OS detected: Please install Intel Power Gadget to measure CPU

[codecarbon WARNING @ 16:53:59] No CPU tracking mode found. Falling back on CPU constant mode.


100 utilisateurs/minute : 2.0746997100e-04 kgCO2


### Test avec 1000 utilisateurs/minute

In [7]:
co2_energy_1000 = run_users_test(1000)
print(f"1000 utilisateurs/minute : {sci(co2_energy_1000)} kgCO2")

[codecarbon WARNING @ 16:56:25] Multiple instances of codecarbon are allowed to run at the same time.
[codecarbon WARNING @ 16:56:27] No CPU tracking mode found. Falling back on estimation based on TDP for CPU. 
 Windows OS detected: Please install Intel Power Gadget to measure CPU

[codecarbon WARNING @ 16:56:27] No CPU tracking mode found. Falling back on CPU constant mode.


1000 utilisateurs/minute : 2.3857472970e-03 kgCO2


### Test avec 1000 utilisateurs/minute + cache

In [8]:
co2_energy_1000_cache = run_users_test(1000, use_cache=True)
print(f"1000 utilisateurs/minute + cache : {sci(co2_energy_1000_cache)} kgCO2")

[codecarbon WARNING @ 17:19:36] Multiple instances of codecarbon are allowed to run at the same time.
[codecarbon WARNING @ 17:19:37] No CPU tracking mode found. Falling back on estimation based on TDP for CPU. 
 Windows OS detected: Please install Intel Power Gadget to measure CPU

[codecarbon WARNING @ 17:19:37] No CPU tracking mode found. Falling back on CPU constant mode.


1000 utilisateurs/minute + cache : 4.1472210234e-03 kgCO2


## Résultats

Cette section affiche les résultats des tests sous forme de tableau pour une comparaison facile.

In [9]:
header = ["Testcase", "Énergie (kgCO2)"]
data = [
    ["10 utilisateurs/minute", f"{sci(co2_energy_10)}"],
    ["100 utilisateurs/minute", f"{sci(co2_energy_100)}"],
    ["1000 utilisateurs/minute", f"{sci(co2_energy_1000)}"],
    ["1000 utilisateurs/minute + cache", f"{sci(co2_energy_1000_cache)}"]
]

print(tabulate(data, headers=header, tablefmt="github"))


| Testcase                         |   Énergie (kgCO2) |
|----------------------------------|-------------------|
| 10 utilisateurs/minute           |       1.74424e-05 |
| 100 utilisateurs/minute          |       0.00020747  |
| 1000 utilisateurs/minute         |       0.00238575  |
| 1000 utilisateurs/minute + cache |       0.00414722  |
